In [146]:
import os
import librosa
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
from keras.models import load_model 


In [147]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [148]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

# Define the model
model = Sequential([
    Input(shape=(100,)),  # Define the input shape here
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])


In [149]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model("Trained_model.h5")


In [150]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])


In [151]:
classes = ['blues', 'classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']


## Preprocessing before feeding it to the model

In [152]:
# Load and preprocess audio data
def load_and_preprocess_data(file_path, target_shape=(150, 150)):
    data = []
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    # Perform preprocessing (e.g., convert to Mel spectrogram and resize)
    # Define the duration of each chunk and overlap
    chunk_duration = 4  # seconds
    overlap_duration = 2  # seconds
                
    # Convert durations to samples
    chunk_samples = chunk_duration * sample_rate
    overlap_samples = overlap_duration * sample_rate
                
    # Calculate the number of chunks
    num_chunks = int(np.ceil((len(audio_data) - chunk_samples) / (chunk_samples - overlap_samples))) + 1
                
    # Iterate over each chunk
    for i in range(num_chunks):
                    # Calculate start and end indices of the chunk
        start = i * (chunk_samples - overlap_samples)
        end = start + chunk_samples
                    
                    # Extract the chunk of audio
        chunk = audio_data[start:end]
                    
                    # Compute the Mel spectrogram for the chunk
        mel_spectrogram = librosa.feature.melspectrogram(y=chunk, sr=sample_rate)
                    
                #mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
        mel_spectrogram = tf.image.resize(tf.convert_to_tensor(np.expand_dims(mel_spectrogram, axis=-1), dtype=tf.float32),target_shape)
        data.append(mel_spectrogram)
    
    return np.array(data)

In [153]:
from IPython.display import Audio
file_path = "./test_rock.wav"
y, sr = librosa.load(file_path, sr=44100)
Audio(data=y, rate=sr)

In [154]:
file_path = "test_rock.wav"
X_test = load_and_preprocess_data(file_path)

In [155]:
X_test.shape

(10, 150, 150, 1)

## Model Prediction

In [156]:
def model_prediction(X_test):
    y_pred = model.predict(X_test)
    predicted_categories = np.argmax(y_pred,axis=1)
    unique_elements, counts = np.unique(predicted_categories,return_counts=True)
    max_count = np.max(counts)
    max_elements = unique_elements [counts == max_count]
    return max_elements[0]

In [157]:
c_index = model_prediction(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step


In [158]:
c_index

9

In [159]:
print("Music Genre :: Predicted Genre ---->",classes[c_index])

Music Genre :: Predicted Genre ----> rock
